# **Launch Site Location Analysis with Folium**

## **SpaceX  Falcon 9 First Stage Landing Prediction**

Launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.

Here, we will be performing interactive visual analytics using Python's `Folium` package.

## Table of Contents

* [Marking all launch sites on a map](#Marking_sites)
* [Mark the success/failed launches for each site on the map](#Mark_launches)
* [Calculating the distances between a launch site and its proximities](#Distances_proximities)

Let's first import required Python packages `Folium` and `Pandas`. The first might have to be installed beforehand.

In [ ]:
#Installing Folium (if needed)
#!pip3 install folium

In [1]:
# Importing the packages
import folium
import pandas as pd

# Importing folium MarkerCluster plugin
from folium.plugins import MarkerCluster

# Importing folium MousePosition plugin
from folium.plugins import MousePosition

# Importing folium DivIcon plugin
from folium.features import DivIcon

## Marking all launch sites on a map <a id='Marking_sites'></a>


Let's add each site's location on a map using each site's latitude and longitude coordinates.

The `dataset_part_2.csv` file obtained in _The Problem and The Approach_ section already includes the latitude and longitude added for each site. We will start by loading this data frame to the object called `spacex_df`.

In [2]:
spacex_df = pd.read_csv('dataset_part_2.csv')
spacex_df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude,Class
0,1,2010-06-04,Falcon 9,8120.677431,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0003,-80.577366,28.561857,0
1,2,2012-05-22,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0005,-80.577366,28.561857,0
2,3,2013-03-01,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B0007,-80.577366,28.561857,0
3,4,2013-09-29,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1.0,0,B1003,-120.610829,34.632093,0
4,5,2013-12-03,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1.0,0,B1004,-80.577366,28.561857,0


Now, we can collect the coordinates for each site by gathering the first launch that took place at each launch site and create the `launch_sites_df` data frame.

In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['FlightNumber','LaunchSite', 'Latitude', 'Longitude', 'Class']]
launch_sites_df = spacex_df.groupby(['LaunchSite'], as_index=False).first()
launch_sites_df.set_index('LaunchSite',inplace=True)
launch_sites_df

,FlightNumber,Latitude,Longitude,Class
LaunchSite,,,,
CCSFS SLC 40,1,28.561857,-80.577366,0
KSC LC 39A,27,28.608059,-80.603956,1
VAFB SLC 4E,4,34.632093,-120.610829,0


<a id='coordinate_notation'></a>
The sign of the above coordinates has a well-defined meaning:
* _Negative_ latitude means _South_ of Equator, whereas _positive_ latitude means _North_ of Equator;
* _Negative_ longitude means _West_ of Greenwich Meridian, whereas _positive_ longitude means _East_ of Greenwich Meridian.

We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [4]:
# Start location is NASA Johnson Space Center, which is described in the 'nasa_coordinate' list
nasa_coordinate = [29.559684888503615, -95.0830971930759] # Latitude and longitude, respectively
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We can use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. Below we are executing this to highlight NASA Johnson Space Center.

In [5]:
# Create a dark orange circle at NASA Johnson Space Center's coordinate with a dark orange popup label showing its name. 
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a dark orange circle at NASA Johnson Space Center's coordinate with an icon of the same color showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

We can see a small dark orange circle near the city of Houston and we can zoom in to enlarge the circle. In contrast, if we zoom out, this circle will shrink.

Now, let's add a circle for each launch site listed in our data frame `launch_sites_df`. We will create and add a `folium.Circle` and a `folium.Marker` for each launch site on the site map.

In [6]:
# Define a list of coordinates (Latitude and Longitude) for Knowles, Oklahoma (USA), which will be our map center this time.
Knowles_OK_US = [36.87278442297836, -100.19376804792711]

# Initialize the map and center it on Knowles, OK (USA)
site_map1 = folium.Map(location=Knowles_OK_US, zoom_start=5)

For each launch site, we add a `Circle` object based on its geographic coordinates (latitude and longitude). In addition, we add the launch site name as a pop up label.

In [7]:
for index in launch_sites_df.index:
    circle = folium.Circle(tuple(launch_sites_df.loc[index,['Latitude','Longitude']].values), radius=1000, color='#000000', fill=True).add_child(folium.Popup(index))
    marker = folium.map.Marker(tuple(launch_sites_df.loc[index,['Latitude','Longitude']].values), icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % index, ))
    site_map1.add_child(circle)
    site_map1.add_child(marker)
site_map1

The generated map above will show VAFB SLC 4E in USA's West coast and both KSC LC 39A and CCSFS SLC 40 in USA's East coast. These last two are superimposed because they are very close to each other on the map at this scale. We will be able to distinguish them if we zoom the map in.

We can see that all launch sites are close to the coastline.

## Mark the success/failed launches for each site on the map <a id='Mark_launches'></a>


Now, let's try to enhance the map above by adding all launch outcomes for each site.
Let's keep in mind that the data frame `spacex_df` includes the `Class` column from the original data set, which indicates if each launch was successful - `Class=1` - or not - `Class=0`, as we can see in the `spacex_df` data frame below.

In [21]:
spacex_df.head()

,FlightNumber,LaunchSite,Latitude,Longitude,Class,marker_color
0,1,CCSFS SLC 40,28.561857,-80.577366,0,red
1,2,CCSFS SLC 40,28.561857,-80.577366,0,red
2,3,CCSFS SLC 40,28.561857,-80.577366,0,red
3,4,VAFB SLC 4E,34.632093,-120.610829,0,red
4,5,CCSFS SLC 40,28.561857,-80.577366,0,red


Next, let's create markers for all launch records.
If a launch was successful ($Class\ 1$), we use a <font color='green'>green</font> marker and if a launch was failed ($Class\ 0$), we use a <font color='red'>red</font> marker.

Note that a launch only happens in one of the three launch sites, which means many launch records will have the exact same coordinate. We use marker clusters to simplify a map containing many markers on the same coordinates.

In order to add marker clusters, we start by creating a `MarkerCluster` object

In [9]:
marker_cluster = MarkerCluster()

and then we create a new column in the `spacex_df` dataframe called `marker_color` to store the marker colors based on the `Class` value. To do this, we will define a function called `assign_marker_color` and apply it on this data frame.

In [10]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'

# Applying the 'assign_marker_color' function on the 'spacex_df' data frame to create the 'marker_color' column
spacex_df['marker_color'] = spacex_df['Class'].apply(assign_marker_color)
spacex_df.head(10)

,FlightNumber,LaunchSite,Latitude,Longitude,Class,marker_color
0,1,CCSFS SLC 40,28.561857,-80.577366,0,red
1,2,CCSFS SLC 40,28.561857,-80.577366,0,red
2,3,CCSFS SLC 40,28.561857,-80.577366,0,red
3,4,VAFB SLC 4E,34.632093,-120.610829,0,red
4,5,CCSFS SLC 40,28.561857,-80.577366,0,red
5,6,CCSFS SLC 40,28.561857,-80.577366,0,red
6,7,CCSFS SLC 40,28.561857,-80.577366,1,green
7,8,CCSFS SLC 40,28.561857,-80.577366,1,green
8,9,CCSFS SLC 40,28.561857,-80.577366,0,red
9,10,CCSFS SLC 40,28.561857,-80.577366,0,red


Now, we add our marker cluster to our map and then we add a `folium.Marker` to the marker cluster for each launch outcome in the `spacex_df` data frame.

In [11]:
# Add marker_cluster to current site_map
site_map1.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successful or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # Customizing each marker - location, pop up text and icon style - and adding it to the marker cluster on the site map
    # as a marker cluster's child.
    marker = folium.Marker(location=tuple(record[['Latitude','Longitude']].values),
    popup=record['FlightNumber'],
    icon=folium.Icon(color=record['marker_color']))
    marker_cluster.add_child(marker)

site_map1

By September 2022, we can see $26$ launches took place on VAFB SLC 4E and $139$ launches took place on the East coast launch sites. If we zoom in on the East coast, we can see that, of those $139$ launches, $48$ of these launches took place in KSC LC 39A and $91$ launches took place in CCSFS SLC 40.

If we run this analysis at a later time, all these numbers will be the same or larger.

If we zoom in further and click on the number displayed by the marker cluster, we will be able to see the <font color='green'>successful</font> and the <font color='red'>unsuccessful</font> launches, each one represented by a single marker. If we click on a single marker, we will be able to see which flight number it is.

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

# Calculating the distances between a launch site and its proximities <a id='Distances_proximities'></a>

Next, we will explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get the coordinates of the mouse cursor each time it is inside the map. As such, while we are exploring the map, we can easily find the coordinates of any points of interest. The coordinates will show up on the top right, with 5 decimal places each. Their meaning is [the same explained before](#coordinate_notation).

In [12]:
# Add Mouse Position to get the coordinates (Latitude, Longitude) for a mouse position on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map1.add_child(mouse_position)
site_map1

We can zoom in to a given launch site and explore its proximities to see if we can easily find any railway, highway, coastline, etc. We can move the mouse to these points and mark down their coordinates (shown on the top-right) in order to calculate the distance of these points to the launch site.

We will calculate the distance between two points on the map based on their latitude $\lambda$ and longitude $\phi$ values, converted to radians, using the haversine formula. Let's consider two places $P$ and $Q$ on the map, with coordinates $(\lambda_1, \phi_1)$ and $(\lambda_2, \phi_2)$, respectively. Then, if we consider the value 

$a = sin^2 \left(\displaystyle \frac{\lambda_2 - \lambda_1}{2} \right) + cos(\lambda_1) \space cos(\lambda_2) \space sin^2 \left(\displaystyle \frac{\phi_2-\phi_1}{2} \right)$ , the distance $d$ between $P$ and $Q$ will be 

$ d = 2R \space \arctan{\left(\displaystyle \frac{\sqrt{a}}{\sqrt{1 - a}}\right)} $ , where $R$ is the Earth's radius, considered here as $6373.0 \space km$.

We will define the function `calculate_distance` to calculate distances between two places using the haversine formula above.

In [13]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(place1, place2):
    # approximate radius of earth in km
    R = 6373.0
    
    # Converting latitude and longitude values of each position to radians. The values obtained from the map are in degrees.
    # 'place1' and 'place2' are lists in which their first value is the latitude and their second value is the longitude.
    place1 = list(map(radians,place1))
    place2 = list(map(radians,place2))
    
    lat1 = place1[0] # Latitude of 'place1'
    lon1 = place1[1] # Longitude of 'place1'
    lat2 = place2[0] # Latitude of 'place2'
    lon2 = place2[1] # Longitude of 'place2'
    
    #Difference between longitudes
    dlon = lon2 - lon1
    
    #Difference between latitudes
    dlat = lat2 - lat1

    # To simplify the code, we use the differences between latitudes and longitudes 'dlat' and 'dlong', respectively, to apply
    # the haversine formula described above.
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    distance = 2 * R * atan2(sqrt(a), sqrt(1 - a))
    return distance

Here, we will mark down the closest point on the coastline to CCSFS SLC 40 using the `MousePosition` values and calculate the distance between that coastline point and this launch site.

In [14]:
CCSFS_SLC40_location = launch_sites_df.loc['CCSFS SLC 40',['Latitude','Longitude']].to_list()
CCSFS_SLC40_location

[28.5618571, -80.577366]

In [15]:
# find coordinate of the closest coastline and then calculate the distance of a launch site to this point, in km.
# e.g.,: Lat: 28.56334  Lon: -80.56804
nearest_coastline = [28.56262, -80.56789] # obtained from the map above by reading the coordinates of the mouse position when
# placed on the nearest coastline location.
CCSFS_SLC40_location = launch_sites_df.loc['CCSFS SLC 40',['Latitude','Longitude']].to_list()
distance_coastline = calculate_distance(CCSFS_SLC40_location, nearest_coastline)
distance_coastline

0.9296196437228371

After obtaining this distance, let's create a marker using `folium.Marker` to show the distance between CCSFS SLC 40 and the nearest position in the coastline.

In [16]:
# Create and add a folium.Marker on our selected closest coastline point on the map
# Display the distance between nearest coastline point and launch site using the icon property 
distance_marker = folium.Marker(
    nearest_coastline,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
)

We can also draw a line between CCSFS SLC 40 to the selected coastline point using `PolyLine`.

In [17]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
lines=folium.PolyLine(locations=[CCSFS_SLC40_location,nearest_coastline], weight=3)
site_map1.add_child(lines)
site_map1.add_child(distance_marker)

The updated map with the distance line, after zooming in on the USA's East coast, should look like the following screenshot:

![Title](Distance%20marker%20CCSFS%20SLC%2040.png)

Similarly, we can draw a line between a launch site to its closest city, railway, highway, etc. We need to to find their coordinates on the map first using `MousePosition`.


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


In [18]:
# Create a marker with distance to a closest city, railway, highway, etc. The coordinates were found by using the mouse cursor
# and recording the coordinates appearing on the top right of the interactive map above.
closest_railway = (28.57205, -80.58527)
closest_highway = (28.56244, -80.57066)
closest_city = (28.40114, -80.60429)

distance_railway = calculate_distance(CCSFS_SLC40_location, closest_railway)
distance_highway = calculate_distance(CCSFS_SLC40_location, closest_highway)
distance_city = calculate_distance(CCSFS_SLC40_location, closest_city)

distance_marker_railway = folium.Marker(
    [28.57204, -80.58541],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#55ff55;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
        )
)

distance_marker_highway = folium.Marker(
    closest_highway,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#e43b3b;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway),
        )
)

distance_marker_city = folium.Marker(
    closest_city,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#cb95e4;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city),
        )
)

In [19]:
# Drawing a line between each marker and the launch site and adding the lines to the previous map
line_railway=folium.PolyLine(locations=[CCSFS_SLC40_location,closest_railway], weight=3, color='#55ff55')
line_highway=folium.PolyLine(locations=[CCSFS_SLC40_location,closest_highway], weight=3, color='#e43b3b')
line_city=folium.PolyLine(locations=[CCSFS_SLC40_location,closest_city], weight=3, color='#cb95e4')

# Adding the lines and the distance markers to the map
site_map1.add_child(line_railway)
site_map1.add_child(line_highway)
site_map1.add_child(line_city)
site_map1.add_child(distance_marker_railway)
site_map1.add_child(distance_marker_highway)
site_map1.add_child(distance_marker_city)

The map, after we zoom in around the Cape Canaveral Space Force Station, in the USA's East Coast, shall look similar to this:

![Title](Distance%20markers%20CCSFS%20SLC%2040.png)

We can zoom this map further in and we realize that CCSFS SLC 40 is actually very close to several lines of communication - it is $660 \ m$ away from the closest highway and $1.37 \ km$ away from the closest railway. This short distance to highways and railways also applies to the other launch sites and it might facilitate the transportation of necessary equipment involved in the launches.

Moreover, we can see that, in general, launch sites are close to the coastline, being it in the West or in the East side of the USA. Specifically, CCSFS SLC 40 is $930 \ m$ away from the coastline. Localizing the launch sites close to the coastline may bring a safe net in case of any unsuccessful landing, as these can eventually take place in the ocean, thereby minimizing the risk to human lives. For a similar reason, launch sites are as far away as possible from cities and other largely populated areas. We can see here that the closest city to the **CCSFS SLC 40** launch site is *Cape Canaveral*, which is $18.07 \ km$ away.

## Acknowledgments

[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01)


Joseph Santarcangelo

This notebook was based on the [Applied Data Science Capstone](https://www.coursera.org/learn/applied-data-science-capstone?specialization=ibm-data-science) course from the [IBM Data Science Professional Certificate](https://www.coursera.org/professional-certificates/ibm-data-science) specialization. However, all descriptions were widely extended and code was improved from the location analysis performed in the course.